In [1]:
import numpy as np
import random



In [2]:

grid_size = 100
start = (0, 0)
goal = (99, 99)
obstacle_percentage = 0.2  

In [3]:

# Create the grid environment
grid = np.zeros((grid_size, grid_size))

# Randomly place obstacles
num_obstacles = int(obstacle_percentage * grid_size * grid_size)
for _ in range(num_obstacles):
    x, y = random.randint(0, grid_size-1), random.randint(0, grid_size-1)
    grid[x, y] = -1  # -1 represents an obstacle

# Ensure start and goal are not obstacles
grid[start] = 0
grid[goal] = 0

# Define MDP parameters
actions = [(0, 1), (1, 0), (0, -1), (-1, 0)]  # Right, Down, Left, Up
gamma = 0.9  # Discount factor
reward_goal = 10
reward_step = -0.1
reward_obstacle = -1


In [4]:

values = np.zeros((grid_size, grid_size))

def is_valid_state(state):
    x, y = state
    return 0 <= x < grid_size and 0 <= y < grid_size and grid[x, y] != -1


In [5]:

def get_next_state(state, action):
    x, y = state
    dx, dy = action
    next_state = (x + dx, y + dy)
    return next_state if is_valid_state(next_state) else state


In [6]:

# Value Iteration
def value_iteration(threshold=1e-4):
    while True:
        delta = 0
        for x in range(grid_size):
            for y in range(grid_size):
                state = (x, y)
                if state == goal:
                    continue  

                v = values[state]
                values[state] = max(
                    reward_step + gamma * values[get_next_state(state, a)]
                    for a in actions
                )
                delta = max(delta, abs(v - values[state]))

        if delta < threshold:
            break

# Q-learning parameters
q_values = np.zeros((grid_size, grid_size, len(actions)))
alpha = 0.1  # Learning rate
epsilon = 0.1  # Exploration rate
num_episodes = 10000


In [7]:

def q_learning():
    for episode in range(num_episodes):
        state = start
        while state != goal:
            if random.uniform(0, 1) < epsilon:
                action_idx = random.randint(0, len(actions) - 1)  # Explore
            else:
                action_idx = np.argmax(q_values[state[0], state[1]])  # Exploit

            action = actions[action_idx]
            next_state = get_next_state(state, action)

            reward = reward_goal if next_state == goal else reward_step
            q_values[state[0], state[1], action_idx] += alpha * (
                reward + gamma * np.max(q_values[next_state[0], next_state[1]]) -
                q_values[state[0], state[1], action_idx]
            )
            state = next_state

# Run Value Iteration and Q-learning, then compare
print("Running Value Iteration...")
value_iteration()
print("Value Iteration completed.")
print("Running Q-learning...")
q_learning()
print("Q-learning completed.")


Running Value Iteration...
Value Iteration completed.
Running Q-learning...
Q-learning completed.


In [8]:

def print_optimal_policy():
    policy_grid = np.zeros((grid_size, grid_size), dtype=str)
    for x in range(grid_size):
        for y in range(grid_size):
            if (x, y) == goal:
                policy_grid[x, y] = "G"
            elif grid[x, y] == -1:
                policy_grid[x, y] = "X"
            else:
                best_action = np.argmax(q_values[x, y])
                if best_action == 0:
                    policy_grid[x, y] = "→"
                elif best_action == 1:
                    policy_grid[x, y] = "↓"
                elif best_action == 2:
                    policy_grid[x, y] = "←"
                elif best_action == 3:
                    policy_grid[x, y] = "↑"
    print(policy_grid)

print("Optimal Policy (Q-learning):")
print_optimal_policy()

Optimal Policy (Q-learning):
[['→' '↓' '↓' ... '→' '→' '↓']
 ['→' '↓' '→' ... 'X' '↑' '←']
 ['←' '↓' '↓' ... '↓' '↓' '→']
 ...
 ['←' '↓' '→' ... '→' '→' '↓']
 ['↓' '←' '↑' ... 'X' '→' '↓']
 ['↓' 'X' '←' ... 'X' '→' 'G']]
